# An excursion in dynamic programming

Dynamic Programming is a family of optimization technique when some constraints must be observed. For example, if we are asked to align the strings `"CATS"` and `"SURF"`, most of us will produce the following:

```
CATS
SURF
```

because we were not given any constraints. If, on the other hand, we were asked to aling these two string so that we maximize overlap between same letters, the result would be:

```
CATS
   SURF
```

## What is alignment?

String alignment is the arragement of two strings that increases or accentuates their similarity. For example, consider the strings `"GANDER"` (a male goose) and `"AND"`. An alignment that accentuates their similarity is:

```
"GANDER"
" AND  "
```
To emphasize the spaces added to make the alignment possible, it's best if we use the dash symbol:

```
"GANDER"
"-AND--"
```
When aligned, the two strings have the same length.

In the example above, we started with strings of different lengths. We can also align strings with the same length. For example:

```
"CATS---"
"---SURF"
```
In this case, we started with two strings (`"CATS"` and `"SURF"`) each 4-characters long. To align them, we turned them into 7-characters long, each, so that their only common letter (`S`) will fall into the same spot.


## How to align?

In most alignment problems, we try to match as many letters between the two strings, as possible. Or to minimize the mismatches as much as we can. Here're some alignment examples.

```
GASTRONOMY     CAT---     CRA-NE
-ASTRONOMY     ---DOG     -RAIN-
```

Our preference for letter matching can be quantified by a penalty function. In its simplest form, the function produces no penalty when two characters $x$ and $y$ match, and some penalty when they do not.


$$\alpha_{xy} = \begin{cases}0,\ \text{if}\ x=y \\1,\ \text{if}\ x\neq y\end{cases}$$

For example, if $x=\texttt{G}$ and $y=\texttt{C}$ then $α_\texttt{GC}=1$. If $x=\texttt{G}$ and $y=\texttt{G}$ then $α_\texttt{GG}=0$.

Adding gaps to adjust the length of an aligned string also incurs a penalty; for simplicity let's say $\alpha_\text{gap}=1$.

## Alignment, step by step

Let's look at some of the possible ways to align the strings `"GANDER"` and `"AND"`.
```
"GANDER"   "GANDER"   "GANDER"   "GANDER"   "GANDER"   "GANDER"
"AND---"   "-AND--"   "--AND-"   "---AND"   "-AN-D-"   "-A-ND-"
  (1)        (2)        (3)        (4)        (5)        (6)
```
For alignment (1) above, we have three mismatches: $\alpha_\texttt{GA}=1$, $\alpha_\texttt{AN}=1$, and $\alpha_\texttt{ND}=1$. Also we have three gaps, and the total penalty is  

$$\alpha_\texttt{GA}+\alpha_\texttt{AN}+\alpha_\texttt{ND}+3\alpha_\text{gap} = 6$$

For alignment (2) above, we have 3 matches:  $\alpha_\texttt{AA}=0$, $\alpha_\texttt{NN}=0$, and $\alpha_\texttt{DD}=1$. Also we have three gaps, and the total penalty is  

$$\alpha_\texttt{AA}+\alpha_\texttt{NN}+\alpha_\texttt{DD}+3\alpha_\text{gap} = 3$$

For alignments (3) and (4) above, we have three mismatches and three gaps. The total penalty for each of these alignments is also 6.

For alignment (5) we have

$$
\alpha_\texttt{AA} + \alpha_\texttt{NN} + \alpha_\texttt{ED} +3\alpha_\text{gap} = 0+0+1+3 = 4
$$

And for alignment (6) we have

$$
\alpha_\texttt{AA} + \alpha_\texttt{DN} + \alpha_\texttt{ED} +3\alpha_\text{gap} = 0+1+1+3 = 5
$$

The second alignment has the lowest penalty. This is the best alignment we can obtain between strings `"GANDER"` and `"AND"`.

## Brute force won't work

In the example above, we look at only 6 possible alignments between `"GANDER"` and `"AND"`. There are several more possibilities to arrange `"AND"` interspersed with gaps, to match the length of `"GANDER"`:
```text
AND---     A-ND--     A-ND--     -A-ND-
-AND--     A--ND-     A--ND-     -A--ND
--AND-     A---ND     A---ND     etc
---AND
```
In fact, there are ${m+n}\choose{n}$ posibilities; where $m$ and $n$ are the lengths of strings $X$ and $Y$. In the example above we have ${{6+3}\choose{3}}=84$ possible arrangements. Not a huge number; maybe we can compute the penalty score for each one of them, and find the arrangement with the lowest penalty.

Now imagine if we had strings with lengths $m=100$ and $n=50$. These are not very long strings, but there are ${150}\choose{50}$ ways to arrange them. That's around $13\times 10^{21}$ possible arrangements. If we compute 1 billion arrangements per second, it will take approximately $10^{12}$ seconds to find all the possible penalty scores. That's nearly 32 years. Imagine how long it will take if the strings have thousands of characters. Brute force won't work!

Just to show the exponential size of the problem, let's recall that ${a}\choose{b}$ is the binomial coefficient, defined, for two integers $a$ and $b$, as:

$$
{{a}\choose{b}} = \frac{a!}{b!(a-b)!}
$$

Imagine strings with $m=10,000$ and $n=500$. These values are more typical for strings representing biological information, such as DNA bases. The number of all possible alignments for these strings would be

$$
{{10,500}\choose{500}} = \frac{10,500!}{500!\ \cdot 10,000!}
$$

That's about $1.8\times {10}^{821}$ possible arrangements.

As of 2023, the fastest supercomputer in the world is capableof about ${10}^{18}$ floating point operations per second (FLOPS). If we tasked that computer to compute all the possible alignments for these two strigs, it would take about ${10}^{796}$ years to complete the operation. Considering that the universe is *only* $13.8\times {10}^9$ years old, the brute force computation is simply impossible.


## There's got to be a better way

And it was found here, in Chicago!

No matter how you aling two strings, there is something remarkable about the last column of their alignment. Consider the following three alignments:

```text
CAT     ---CAT     CAT---
DOG     DOG---     ---DOG

(i)      (ii)       (iii)
```

None make sense -- there are no common letters between the two strings, but notice the last column. It is always in one of these three states:

(i) The last column has the last characters of each string;

(ii) the last column that the last character of the first string and a gap;

(iii) the last column has a gap and the last character of the second string.

Let's observe these cases with three more suitable strings:

```text
--CYCLE     TEAM     SEA-
BICYCLE     TEA-     SEAL
  (i)       (ii)     (iii)
```   

Alignment (i) above is the best option we have to align strings `CYCLE` and `BICYCLE`. Alignments (ii) is the best match between `TEAM` and `TEA`. And alignment (iii) is the best match between `SEA` and `SEAL`.

Let's focus on alignment (i) and in particular its last column:

```
--CYCL E
BICYCL E
```

If we remove the last column, we notice that the remaining strings are also nicely aligned:

```
--CYC L
BICYC L
```

It can be proved that if the last column contains the last characters of the optimally aligned strings, the remaining strings after we remove the last column, are also aligned optimally.

To formalize things, $X_m=(x_1, x_2, \ldots, x_m)$ and $Y_n= (y_1, y_2, \ldots, y_n)$ are strings with $m$ and $n$ symbols respectively. Their optimal alignment is two strings of **equal** length whose symbols match each other as much as possible. To achieve this matching it may be necessary to insert gaps in $X$ or $Y$. For example, if $X = (c,y,c,l,e)$ and $Y=(b,i,c,y,c,l,e)$, their optimal alignment is the strings:
\begin{align}
\bar{X} = (-,-,c,y,c,l,e) \\
\bar{Y}=(b,i,c,y,c,l,e)
\end{align}

Notice that the aligned strings $\bar{X}$ and $\bar{Y}$ have equal lengths, $\bar{m}=\bar{n}$. In general, $\bar{m}= \bar{n} \geq \max{(m,n)}$.


## A touch of formalism

Let's take an even closer look at the alignment of strings $X_7=(\text{b}, \text{i}, \text{c}, \text{y}, \text{c}, \text{l}, \text{e} )$ and $Y_5=( \text{c}, \text{y}, \text{c}, \text{l}, \text{e})$. The last column contains the characters
$\left|\begin{array}{c}\text{e}\\ \text{e}\end{array}\right|$, the last letters of the words *cycle* and *bicycle*. In more abstract terms, the last column is
$\left|\begin{array}{c}x_7\\ y_5\end{array}\right|$.

When the last column includes the last letters of the input strings, we  can write the aligned strings $\bar{X}$ and $\bar{Y}$ as
\begin{align}
\bar{X} = X'\ & x_m \\
\bar{Y} = Y'\ & y_n \\
\end{align}

In the expression above, $X'$ and $Y'$ are the aligned strings without the last characters of the input strings; in this case:
\begin{align}
\bar{X_7} = X_6\ & x_7 \\
\bar{Y_5} = Y_4\ & y_5 \\
\end{align}

When the alignment ends with the last characters of the input strings ($X$ and $Y$), the substrings $X'$ and $Y'$, as defined above, are also optimally aligned. And in fact, $X'$ and $Y'$ are an optimal alignment of substrings $X_{m-1}= (x_1, x_2, \ldots, x_{m-1})$ and $Y_{n-1}=(y_1, y_2, \ldots, y_{n-1})$.
A proof of this is included in the Miscelaneous section at the bottom of this notebook.

This is good news. Because when the last column of the alignment contains the last letters of the strings
$X_m$ and $Y_n$
then removing the last column gives an optimal alignment of the strings
$X_{m-1}$
and
$Y_{n-1}$.

## A bit more formalism

So far, we discussed what happens when the last column in an optimal alignment contains the last characters of the strings we want to align. There are two other possible arrangements for that last column: it could be the last character of the first string, over a gap , as in

$$\boxed{\begin{array}{c}\texttt{TEAM}\\ \texttt{TEA-}\end{array}}$$

Or, the last column may contain a gap over the last letter of the second string, as in

$$\boxed{\begin{array}{c}\texttt{SEA-}\\ \texttt{SEAL}\end{array}}$$

### The case of $x_m$ over a gap in last column

In this case, the last column of the optimal alignment contains the last character of the first string over a gap. We can write

\begin{align}
\bar{X} = X'\ & x_m \\
\bar{Y} = Y\  &- \\
\end{align}

When the last column  $\left|\begin{array}{c}x_m\\ -\end{array}\right|$ is removed, the rest of the alignment is an optimal alignment of $X'$ and $Y$. In other words, an optimal alignment between substring $X_{m-1}=(x_1, x_2, \ldots, x_{m-1})$ and the full second string $Y_n=(y_1, y_2, \ldots, y_{n})$.

The proof is in the Miscelaneous section of this notebook.


### The case of a gap over $y_n$ in last column

In this case, the last column of the optimal alignment contains a gap over the last character of the second string. We can write

\begin{align}
\bar{X} = X\ & - \\
\bar{Y} = Y'\ &y_n \\
\end{align}

When the last column  $\left|\begin{array}{c}-\\ y_n\end{array}\right|$ is removed, the rest of the alignment is an optimal alignment of $X$ and $Y'$. In other words, an optimal alignment between the full string $X_m=(x_1, x_2, \ldots, x_{m})$ and the substring $Y_{n-1}=(y_1, y_2, \ldots, y_{n-1})$.

The proof is similar to the case of  $\left|\begin{array}{c}x_m\\ -\end{array}\right|$ shown in the Miscelaneous section later in this notebook.

## Putting the three cases together

Each of the three cases we reviewed above, tells us something about the strings that remain when we remove the last column in an optimal alignment.


*   When the last column is $\left|\begin{array}{c}x_m\\ y_n\end{array}\right|$, the remaining alignment is an optimal alignment of substrings $X_{m-1}$ and $Y_{n-1}$.
<br>

*   When the last column is $\left|\begin{array}{c}x_m\\ -\end{array}\right|$, the remaining alignment is an optimal alignment of substring $X_{m-1}$ and $Y_{n}$.
<br>

*   When the last column is $\left|\begin{array}{c}-\\ y_n\end{array}\right|$, the remaining alignment is an optimal alignment of $X_{m}$ and substring $Y_{n-1}$.

This is important! When we are looking at an optimal alignment of two strings, we know that we got there only through one of the scenarios above. These scenarios provide us with a way to build the optimal alignment of two strings, from previous alignments.

Consider the substrings $X_i$ and $Y_j$ that comprise the first $i$ and $j$ symbols of $X$ and $Y$ respectively. Let $P_{i,j}$ be the total penalty of the optimal alignment of these substrings.

The scenarios earlier tell us that there are three ways to achieve an optimal alignment for $X_i$ and $Y_j$:

1. from the optimal alignment of $X_{i-1}$ and $Y_{j-1}$ to which we'll add one more column with symbols $x_i$ and $y_j$; or,
1. from the optimal alignment of $X_{i-1}$ and $Y_j$ to which we'll add one more column with $x_i$ and a gap; or,
1. from the optimal alignment of $X_i$ and $Y_{j-1}$ to which we'll add one more column with a gap and $y_j$.


Of these three choices, we select the one with the smallest penalty:

\begin{align}
P_{i,j} = \min{\left( P_{i-1,\ j-1} + \alpha_{x_i y_j},\ \
                  P_{i-1,\ j} + \alpha_\text{gap},\ \
                  P_{i,\ j-1} + \alpha_\text{gap}
  \right)}
\end{align}


All that remains is to work out some base cases, specifically how to align the first two symbols of strings $X$ and $Y$. Their optimal alignment penalty is given by

\begin{align}
P_{1,1} = \min{\left( P_{0,\ 0} + \alpha_{x_1 y_1},\ \
                  P_{0,\ 1} + \alpha_\text{gap},\ \
                  P_{1,\ 0} + \alpha_\text{gap}
  \right)}
\end{align}

If the smallest of these three quantities is $P_{0,\ 0} + \alpha_{x_1 y_1}$, then we align the first letters of the strings together; for example, strings `SEA` and `SEAL` fall in this case.

If the smallest quantity is $P_{0,\ 1} + \alpha_\text{gap}$ or $P_{1,\ 0} + \alpha_\text{gap}$ then it's the first letter from one string and a gap; this would be the case for strings `SEAT` and `EAT`.

The penalties $P_{0,0}$, $P_{0,1}$ and $P_{1,0}$ are easy to compute.

* $P_{0,0}$
is the total penalty for the optimal alignment of two empty strings, i.e., strings with length 0. The are no characters to match nor gaps to insert. Therefore, there are no $\alpha_{xy}$ and $\alpha_\text{gap}$ penalties to add. $P_{0,0}=0$.

* $P_{0,1}$
is the total penalty for the optimal alignment of an empty string with a string that has one symbol. The are no characters to match, only one gap to insert so that both strings will have the same length.  $P_{0,1}=\alpha_\text{gap}$. And in general, $P_{0,j} = j\alpha_\text{gap}$. Because the alignment of an empty string with a string of length $j$ requires the insertion of $j$ gaps in the empty string.

* Likewise, $P_{i,0} = i\alpha_\text{gap}$.

Only one of the three paths above is used to arrive to an *optimal* $P_{i,j}$. We use the ``min`` function to tell us which one.

![](https://drive.google.com/uc?export=view&id=1fUsxstQEq0NY-GWQQbZWi2HbmmN9Iz-L)

<br/>

\begin{align}
P_{i,j} = \min{( \underbrace{P_{i-1,\ j-1} + \alpha_{x_i y_j}}_{\begin{array}{c}\text{optimal alignment}\\\text{of}\ X_{i-1}\ \text{and}\ Y_{j-1}\\\text{with}\ \left|\begin{array}{}x_i\\y_j\end{array}\right|\ \text{added}\\\text{as last column}\\\text{(Scenario 1)}\end{array}},\ \
                  \underbrace{P_{i-1,\ j} + \alpha_\text{gap}}_{\begin{array}{c}\text{optimal alignment}\\\text{of}\ X_{i-1}\ \text{and}\ Y_{j}\\\text{with}\ \left|\begin{array}{c}x_i\\\text{gap}\end{array}\right|\ \text{added}\\\text{as last column}\\\text{(Scenario 2)}\end{array}},\ \
                  \underbrace{P_{i,\ j-1} + \alpha_\text{gap}}_{\begin{array}{c}\text{optimal alignment}\\\text{of}\ X_{i}\ \text{and}\ Y_{j-1}\\\text{with}\ \left|\begin{array}{c}\text{gap}\\y_j\end{array}\right|\ \text{added}\\\text{as last column}\\\text{(Scenario 3)}\end{array}})}
\end{align}

## There and back again

Successive applications of the formula above lead us from $P_{1,1}$ to $P_{m,n}$, computing the total penalty for the best alignment possible. But we don't quite know what that alignment looks like. For example, when aligning `BICYCLE` and `CYCLE`,

$$
P_{1,1} = \min\{
  P_{0,0}+\alpha_{\texttt{B}\texttt{C}},\
  P_{1,0}+\alpha_{\text{gap}},\
  P_{0,1}+\alpha_{\text{gap}}
  \}
$$

Using

\begin{align}
\alpha_\text{gap} & = 1,\ \text{and} \\
\alpha_{xy} & = \begin{cases}0,\ \text{if}\ x=y \\1,\ \text{if}\ x\neq y\end{cases}\\ \\
\end{align}

This produces the result:
$$
P_{1,1} = \min\{0+1,\ 0+1,\ 0+1\} = 1
$$
whose provenance is inconclusive. The ambiguity may persist through to $P_{m,n}$. We may now how "expensive" the optimal alignment may be (that's what the value of $P_{m,n}$ is), but we don't know which path led us from $P_{1,1}$ to $P_{m,n}$.

To demonstrate the ambiguity, let's consider all possible $P_{i,j}$
values for strings `cycle` and `bicycle`. We know that the best alignment is obtained by placing two gaps in front of `cycle`. The total penalty will be $2\alpha_\text{gap}$. That's the value at the bottom right box: the cost of two gaps. Just by looking at that value, however, doesn't tell us where the gaps fall.

![](https://drive.google.com/uc?export=view&id=1ctxVDetBRfdNa8aRGdo8DjjBqqEHOI-b)

Once we compute all the $P_{i,j}$ values, we need to backtrack our steps to find how to align the strings. We start with the bottom right corner of the penalty table $P$, in this case $P_{5,7}$. That's the the total penalty we pay for the optimal alignment of the two strings (the cost of two gaps). But how did we get there?


There are three scenarios, originating from the surrounding cells in the table:

![](https://drive.google.com/uc?export=view&id=1d5WAuuxTN0egZFOn760vJHwcrNN7oHaU)

These cells correspond to the penalties for other scenarios. For example, the alignment
```
bicycl-
--cycle
```
has 3 gaps and therefore a cost of $P_{5,6}=3\alpha_\text{gap}$ (and since $\alpha_\text{gap}=1$, the cost is 3). The last column we could add to this alignment is $\left|\begin{array}{c}\texttt{e}\\ \texttt{-}\end{array}\right|$. Because it includes an extra gap, the total penalty for the final result
```
bicycl-e
--cycle-
```
will be $P_{5,6}+\alpha_\text{gap}=4\alpha_\text{gap}$. This cannot be the path that leads to the final penalty of 2.

Similarly, the alignment
```
bicycle
--cycl-
```
when completed with the last column $\left|\begin{array}{c}\texttt{-}\\ \texttt{e}\end{array}\right|$, will also be $4\alpha_\text{gap}$ and cannot be the path that leads to the final penalty of 2.

Finally, we have the alignment
```
bicycl
--cycl
```
with a penalty of $P_{4,6}=2\alpha_\text{gap}$. The last column we can add to this alignment is $\left|\begin{array}{c}\texttt{e}\\ \texttt{e}\end{array}\right|$. Total penalty for
```
bicycl e
--cycl e
```
will be $P_{5,7}=P_{4,6}+\alpha_{\texttt{ee}} =2\alpha_\text{gap}$, because $\alpha_{\texttt{ee}}=0$.

By definition,

\begin{align}
P_{5,7} = \min\{
  & P_{5,6}+\alpha_\text{gap},\ &\textsf{if we added column}\ \left|\begin{array}{c}\texttt{e}\\ \texttt{-}\end{array}\right| \\ \\
  & P_{4,7}+\alpha_\text{gap},\ &\textsf{if we added column}\ \left|\begin{array}{c}\texttt{-}\\ \texttt{e}\end{array}\right| \\ \\
  & P_{4,6}+\alpha_\texttt{ee}\ &\textsf{if we added column}\ \left|\begin{array}{c}\texttt{e}\\ \texttt{e}\end{array}\right| \\
  \}
\end{align}

and the only scenario that fulfills the $\min$ constraint above the addition of column $\left|\begin{array}{c}\texttt{e}\\ \texttt{e}\end{array}\right|$.

Therefore, we arrived at $P_{5,7}$ from $P_{4,6}$. Now we repeat the process to tell how we arrive to $P_{4,6}$, and so on.

# Your assignment

Using the information here, implement the alignment algorithm in two steps.

First compute the penalties table $P_{i,j}$ for input strings $X$ and $Y$ of lengths $m$ and $n$ respectively.

Second, beginning from the bottom-right cell $P_{m,n}$ find the last columns, reconstrucing the optimal alignm from back to front. Using the example with `cycle` and `bicyle`, backtracking $P$ will produce the following end columns:

$$
\left|\begin{array}{c}\texttt{e}\\ \texttt{e}\end{array}\right|,\
\left|\begin{array}{c}\texttt{l}\\ \texttt{l}\end{array}\right|,\
\left|\begin{array}{c}\texttt{c}\\ \texttt{c}\end{array}\right|,\
\left|\begin{array}{c}\texttt{y}\\ \texttt{y}\end{array}\right|,\
\left|\begin{array}{c}\texttt{c}\\ \texttt{c}\end{array}\right|,\
\left|\begin{array}{c}\texttt{i}\\ \texttt{-}\end{array}\right|,\
\left|\begin{array}{c}\texttt{b}\\ \texttt{-}\end{array}\right|
$$

The starter code below uses different values for $\alpha_\text{gap}$ and $\alpha_{xy}$ when $x\neq y$. In our example both gap and misalignment penalties were the same ($\alpha_\text{gap}=\alpha_{x\neq y}=1$). By using different values for these penalties we declare a preference for one over the other.

# Reading material

* **PDF:** [Dynamic Programming](https://jeffe.cs.illinois.edu/teaching/algorithms/book/03-dynprog.pdf) from Jeff Erickson's *Algorithms*.

* **PDF:** [History and development of dynamic programming](https://drive.google.com/file/d/1dDh0Tp37r4sRPWSuyKYxF6WFssRJEfb9/view?usp=drive_link) from the IEEE Control Systems Magazine ( Volume: 4, Issue: 4, November 1984), published as a memorial to Richard E Bellman, the creator of dynamic programming.

* **Web:** [Wikipedia entry for the Needleman-Wunch algorithm](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm).

* **PDF:** [Scanned copy of the 1970 Needleman-Wunch paper](https://drive.google.com/file/d/1A_ubZtpSRRvO4BCEfgWkbMVuhlNM5c8E/view?usp=drive_link).


In [ ]:
# Test strings
X = 'CYCLE'
Y = 'BICYCLE'

a_gap = 1

def score(x,y):
    """Naive match/mismatch scoring a_{xy} = 0 if x==y, 2 otherwise"""
    return 0 if x == y else 2

In [ ]:
def alignment_penalties(x, y):
    # Define the dimensions of the matrix based on the size of x and y
    n, m = len(x), len(y)

    # Create penalties table
    P = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first row and column with gap penalties
    for i in range(1, n + 1):
        P[i][0] = i * a_gap
    for j in range(1, m + 1):
        P[0][j] = j * a_gap

    # Build the penalities table by taking the minimum of the values from
    # the left, above, and diagonal. These values build off each other.
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            P[i][j] = min(P[i-1][j-1] + score(x[i-1], y[j-1]), # Diagonal
                          P[i-1][j] + a_gap,                   # left
                          P[i][j-1] + a_gap)                   # up

    # Return the penalties tables
    return P

In [ ]:
def backtrack(x, y, P):
    # Initialize the result array which will be a list of tuples
    res = []

    # Initialize the i and j values, to the end of the strings
    i = len(x)
    j = len(y)

    # keep looping until we've gotten the optimal string alignment
    while i > 0 or j > 0:
        # Move Diagonal (up and left) if we haven't ran out of string
        if i > 0 and j > 0:
            diagonal = P[i-1][j-1] + score(x[i-1], y[j-1])
        else:
            diagonal = float('inf')

        # Move Left if we haven't ran out of string
        if j > 0:
            left = P[i][j-1] + a_gap
        else:
            left = float('inf')

        # Move up if we haven't ran out of string
        if i > 0:
            up = P[i-1][j] + a_gap
        else:
            up = float('inf')

        # Get the minimum value so that we can check where we need to go
        min_value = min(diagonal, left, up)

        if diagonal == min_value:
            # Store the value to add to the from the diagonal
            move_value = ((x[i-1], y[j-1]))
            # Move the i and j values up and to the left (diagonally)
            # So that we can keep backtracking
            move_i = 1
            move_j = 1
        elif left == min_value:
            # Store the value to add to the from the left
            move_value = ("-", y[j-1])
            # Move the j value to the left
            # So that we can keep backtracking
            move_i = 0
            move_j = 1
        elif up == min_value:
            # Store the value to add to the from the left
            move_value = (x[i-1], "-")
            # Move the i value to the up
            # So that we can keep backtracking
            move_i = 1
            move_j = 0
        # Add the value to the result array and move
        res.append(move_value)
        i -= move_i
        j -= move_j

    # return the optimal string alignment
    return res

In [ ]:
def dp(x, y):
    ''' Function to return the optimal alignment of strings x and y
        This is basically just a wrapper for both the assigned functions
        but changing the return into an aligned string'''
    # get the penalties table for string x and y
    P = alignment_penalties(x, y)

    # Get the optimal alignment via backtracking
    alignment = backtrack(x, y, P)

    # return the value as a aligned string ontop of each other
    s1 = ''.join(x[0] for x in reversed(alignment))
    s2 = ''.join(x[1] for x in reversed(alignment))
    return f"{s1}\n{s2}"


In [ ]:
## Sample use case

# get the penalties table for string x and y
P = alignment_penalties(X, Y)

for row in P:
    print(row)

print()

# Get the optimal alignment via backtracking
alignment = backtrack(X, Y, P)

for elm in alignment:
    print(elm)

print()

# Run the DP method to print the aligned string ontop of each other
print(dp(X, Y))


[0, 1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 2, 3, 4, 5, 6]
[2, 3, 4, 3, 2, 3, 4, 5]
[3, 4, 5, 4, 3, 2, 3, 4]
[4, 5, 6, 5, 4, 3, 2, 3]
[5, 6, 7, 6, 5, 4, 3, 2]

('E', 'E')
('L', 'L')
('C', 'C')
('Y', 'Y')
('C', 'C')
('-', 'I')
('-', 'B')

--CYCLE
BICYCLE


# Miscellaneous

## Proofs

### Optimal alignment with $x_m$ and $y_n$ in last column

**Theorem:** An optimal alignment of $X$ and $Y$ that contains $x_m$ and $y_n$ in its last column, comprises an optimal alignment of $X_{m-1}$ and $Y_{n-1}$ as well.

Here, $X_i$ is the substring with the first $i$ characters of $X$ and $Y_j$ is the substring with the first $j$ characters of $Y$.

**Proof by contradiction:** Let $P$ be the penalty for the optimal alignment of $X_{m-1}x_m$ and $Y_{n-1}y_n$. The penalty for the alignment $X_{m-1}$ and $Y_{n-1}$ is then $P'= P-\alpha_{x_m y_n}$. That's the penalty of the optimal alignment minus the score associated with the match (or mismatch) of $x_m$ and $y_n$ in the last column.

We claim that $P'$ is the minimum penalty for aligning $X_{m-1}$ and $Y_{n-1}$.

Let's assume that there is an other aligment of $X_{m-1}$ and $Y_{n-1}$ whose penalty is $P^* < P'$. If we add the column $\left|\begin{array}{c}x_m\\ y_n\end{array}\right|$ to this *hypothetical* optimal alignment, the total penalty for the new alignment of $X_m$ and $Y_n$ becomes $P^* + a_{x_m y_n}$.

We already agree that $P^* < P'$. The relation is preserved if we add the same number to both parts; here we add $a_{x_m y_n}$:

\begin{align}
P^* + a_{x_m y_n} < P' + a_{x_m y_n}
\end{align}

Substituting $P'=P-a_{x_m y_n}$:

\begin{align}
P^* + a_{x_m y_n} &< (P-a_{x_m y_n}) + a_{x_m y_n} \\
P^* + a_{x_m y_n} &< P
\end{align}

In other words, the aligmnent of  $X_{m-1}$ and $Y_{n-1}$ with penalty $P^*$ augmented by $x_m$ and $y_n$ at its end, has a smaller penalty than that of the optimal alignment of $X$ and $Y$. Which is a contradiction, *QED*.

<br>

### Optimal alignment with $x_m$ over a gap in last column


**Theorem:** if the last column of the optimal alignment of $X$ and $Y$ is $\left|\begin{array}{c}x_m\\ \text{gap}\end{array}\right|$ then the rest of the alignment (when the last column is removed) is an optimal alignment of $X_{m-1}$ and $Y$.

**Proof:** Let $P$ be the total penalty for the optimal alignment of $X$ and $Y$, and $P'$ the penalty of the alignment between $X_{m-1}$ and $Y$. Then $P'=P-a_\text{gap}$. If $P'$ does not correspond to the optimal alignment of $X_{m-1}$ and $Y$, there must be another optimal alignment for these strings with penanlty $P^*<P'=P-\alpha_\text{gap}$. Adding the last column $\left|\begin{array}{c}x_m\\ \text{gap}\end{array}\right|$ to $P^*$ we get: $P^*+\alpha_\text{gap}< P$, i.e., there is an alignment for strings $X$ and $Y$ whose penalty is less than that of their optimal alignment, which is a contradiction. Therefore, the rest of the alignment of $X$ and $Y$ when we remove its last column $\left|\begin{array}{c}x_m\\ \text{gap}\end{array}\right|$ is an optimal alignment for $X_{m-1}$ and $Y$.